In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, f1_score
import gc

In [2]:
# tf.debugging.set_log_device_placement(True)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

In [3]:
window_size = 10
# def build_model():
#     model = tf.keras.Sequential([
#     # data_augmentation,
#     layers.Conv2D(16, 2, padding='same', activation='relu', input_shape= (2*window_size+1,1024,1)),
#     layers.MaxPooling2D(),
#     layers.Conv2D(16, 2, padding='same', activation='relu'),
#     layers.MaxPooling2D(),
#     layers.Flatten(),
#     layers.Dense(16, activation='relu'),
#   ])
#     return model


def build_model():
    model = models.Sequential()
    model.add(layers.Conv1D(filters = 64, kernel_size = 1,activation='relu', 
                      input_shape= (2*window_size+1,1024), padding="same"))
#     model.add(layers.MaxPooling1D(pool_size=3))
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv1D(filters = 128, kernel_size = 1, activation = 'relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Flatten())
    model.add(layers.Dense(8, activation='relu'))
    return model


def get_model():
    model1 = build_model()
    model2 = build_model()
    model1_model2 = layers.concatenate([model1.output, model2.output], name = "concatenated_layer")
    output_layer = layers.Dense(8, activation='relu')(model1_model2)
    output_layer = layers.LeakyReLU(alpha=0.3)(output_layer)
    # output_layer = layers.Dense(16, activation='relu')(output_layer)
    output_layer = layers.Dropout(0.2)(output_layer)
    output_layer = layers.Dense(1, activation='sigmoid')(output_layer)
    model = models.Model(inputs=[model1.input, model2.input], outputs=output_layer, name ="merged_layers")
    return model
# model = get_model()
# model.summary()



In [4]:
model = get_model()
model.summary()

Model: "merged_layers"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 conv1d_input (InputLayer)      [(None, 21, 1024)]   0           []                               
                                                                                                  
 conv1d_2_input (InputLayer)    [(None, 21, 1024)]   0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 21, 64)       65600       ['conv1d_input[0][0]']           
                                                                                                  
 conv1d_2 (Conv1D)              (None, 21, 64)       65600       ['conv1d_2_input[0][0]']         
                                                                                      

In [5]:
def compile_model(model):
  model.compile(optimizer='adam', loss = tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
  return model
# Function to get_score from the model given the training and test dataset

def get_score(model, X_train, X_test, y_train, y_test):
  # patient early stopping
#   es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=10)
#   history = model.fit(X_train, y_train, epochs=10, validation_data = (X_test, y_test), verbose=0, callbacks=[es])
  with tf.device("/gpu:0"):
      history = model.fit(X_train, y_train, epochs=30, batch_size = 2048, validation_data = (X_test, y_test), verbose=0)


  # X_shape = X_train.shape
  #X_train = np.expand_dims(X_train, axis=0)
  # datagen.fit(X_train)
  # fits the model on batches with real-time data augmentation:
  # model.fit(X_train, y_train, batch_size=32,epochs=20)

  train_acc_history = history.history['accuracy']
  val_acc_history = history.history['val_accuracy']
  # evaluate the model
  _, train_acc = model.evaluate(X_train, y_train, verbose=0)
  _, test_acc = model.evaluate(X_test, y_test, verbose=0)
  return train_acc, test_acc

In [6]:

data_bert = np.load('/kaggle/input/data21/data21_bert.npz')
positive_bert = data_bert['positive']
negative_bert = data_bert['negative']
data_T5 = np.load('/kaggle/input/data21/data21_t5.npz')
positive_T5 = data_T5['positive']
negative_T5 = data_T5['negative']

positive_dataset = np.hstack((positive_bert, positive_T5))
negative_dataset = np.hstack((negative_bert, negative_T5))
print(negative_dataset.shape[0])
X = np.vstack([positive_dataset, negative_dataset]) 
p_label = np.ones(positive_dataset.shape[0])
n_label = np.zeros(negative_dataset.shape[0])
Y = np.append(p_label,n_label,axis=0)

del data_bert
del positive_bert
del negative_bert
del data_T5
del positive_T5
del negative_T5
del positive_dataset
del negative_dataset
gc.collect()
print(X.shape)
print(Y.shape)


5183
(10027, 42, 1024)
(10027,)


In [7]:
import sklearn
from sklearn.model_selection import train_test_split
import gc
X_train, X_t, Y_train, Y_t = train_test_split( X, Y, test_size=0.05, random_state=42)
folds = StratifiedKFold(n_splits=10)
scores = {}
scores['train_acc'] = []
scores['test_acc'] = []
max_test_acc = 0
max_mcc = 0
X = X_train
Y = Y_train
X_te = [X_t[:, :21], X_t[:, 21:]]
y_true = Y_t
for train_index, test_index in folds.split(X,Y):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], \
                                    Y[train_index], Y[test_index]
    model = get_model()
    model = compile_model(model)
    X_train = [X_train[:, :21], X_train[:, 21:]]
    X_test = [X_test[:, :21], X_test[:, 21:]]
    train_acc, test_acc = get_score(model, X_train, X_test, y_train, y_test)
    y_pred = model.predict(X_te) > 0.5
    mcc = sklearn.metrics.matthews_corrcoef(y_true, y_pred)
    print("test mcc: ", mcc)
#     if test_acc > max_test_acc:
    if mcc > max_mcc:
        model.save('/kaggle/working/model1.h5')
#         max_test_acc = test_acc
        max_mcc = mcc
    print(train_acc, test_acc)
    scores['train_acc'].append(train_acc)
    scores['test_acc'].append(test_acc)
    del model
    del X_train
    del X_test
    del y_train
    del y_test
    gc.collect()


16/16 [==============================] - 0s 9ms/step
test mcc:  0.5181850832761379
0.9674521684646606 0.7513116598129272
16/16 [==============================] - 0s 5ms/step
test mcc:  0.5555212969555163
0.9245216846466064 0.7701993584632874
16/16 [==============================] - 0s 5ms/step
test mcc:  0.5230288885028722
0.9317545294761658 0.753410279750824
16/16 [==============================] - 0s 4ms/step
test mcc:  0.561899478674152
0.9611526131629944 0.7502623200416565
16/16 [==============================] - 0s 5ms/step
test mcc:  0.5329389268538984
0.8645590543746948 0.7712486982345581
16/16 [==============================] - 0s 4ms/step
test mcc:  0.5728648075369001
0.8788055777549744 0.7584033608436584
16/16 [==============================] - 0s 5ms/step
test mcc:  0.5439277343573953
0.9122827649116516 0.7626050710678101
16/16 [==============================] - 0s 4ms/step
test mcc:  0.5272131666599241
0.9550915956497192 0.7510504126548767
16/16 [===========================

In [8]:
y_true= Y_t
new_model = tf.keras.models.load_model('/kaggle/working/model1.h5')
X_te = [X_t[:, :21], X_t[:, 21:]]
y_pred = new_model.predict(X_te) > 0.5
mcc =sklearn.metrics.matthews_corrcoef(y_true, y_pred)
print(mcc)

16/16 [==============================] - 0s 4ms/step
0.5728648075369001


In [ ]:
f1_score = sklearn.metrics.f1_score(y_true, y_pred)
print("f1 score: ",f1_score)
acc = sklearn.metrics.accuracy_score(y_true, y_pred)
print("acc: ", acc)
confusion_matrix = sklearn.metrics.confusion_matrix(y_true, y_pred)
print("confusion matrix: ",confusion_matrix)


In [14]:
#TSNE plot
from sklearn.manifold import TSNE

# The default of 1,000 iterations gives fine results, but I'm training for longer just to eke
# out some marginal improvements. NB: This takes almost an hour!
tsne = TSNE(random_state=1, n_iter=15000, metric="cosine")
X_mid = X[:,11].reshape((10027,1024))
print(X_mid.shape)
embs = tsne.fit_transform(X_mid)


(10027, 1024)


/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/manifold/_t_sne.py:827: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  FutureWarning,


NameError: name 'df' is not defined

In [ ]:
from matplotlib import pyplot as plt
FS = (10, 8)
fig, ax = plt.subplots(figsize=FS)
# Make points translucent so we can visually identify regions with a high density of overlapping points
ax.scatter(embs[:, 0], embs[:, 1], alpha=.1);